In [16]:
import concurrent.futures
import os
import numpy as np
import pickle
import time

In [17]:
FILEPATHS = []
root_dir="/home/project_x/data/"
scope = 'validation'
root_path = root_dir + scope
for cam in ['FRONT']:
    cam_filepaths = os.listdir(f"{root_path}/{cam}")
    FILEPATHS += [f"{root_path}/{cam}/{i}" for i in cam_filepaths]

In [18]:
len(FILEPATHS)

39988

In [19]:
FILEPATHS[0]

'/home/project_x/data/validation/FRONT/blob_130_frame_124.pickle'

In [20]:
def load_pickle(fpath):
    with open(fpath, "rb") as f:
        return pickle.load(f)

In [21]:
def load_annotations(annot):
        # get ground truth annotations
        annotations = np.zeros((0, 5))

        # some images appear to miss annotations
        if len(annot) == 0:
            return annotations

        # parse annotations
        for idx, a in enumerate(annot):

            # some annotations have basically no width / height, skip them
            if a.box.width < 1 or a.box.length < 1:
                continue

            annotation = np.zeros((1, 5))
            annotation[0, :4] = [
            a.box.center_x -0.5*a.box.length,
            a.box.center_y -0.5*a.box.width,
            a.box.center_x +0.5*a.box.length,
            a.box.center_y +0.5*a.box.width,
            ]
            annotation[0, 4] = a.type-1
            annotations = np.append(annotations, annotation, axis=0)


        return annotations

In [22]:
def get_paths(ran, filepaths=FILEPATHS, exclusions=True,):
    fro, up_to = ran
    # Filter exclusions
    if exclusions is not None:
        filepaths = [
            i for i in filepaths[fro:up_to] if load_annotations(load_pickle(i)['labels'].labels).any()
        ]   
    return filepaths

In [ ]:
if scope == 'validation':
    start = time.perf_counter()
    with concurrent.futures.ThreadPoolExecutor(max_workers = 8) as executor:
        args1 = range(0, 39000, 1000)
        args2 = range(1000, 40000, 1000)
        args = list(zip(args1, args2))
        args3 = [(39000, 39989)]
        args += args3
        fs = executor.map(get_paths, args)
    finish = time.perf_counter()
    print(f'{finish - start}')

elif scope == 'train':
    start = time.perf_counter()
    with concurrent.futures.ThreadPoolExecutor(max_workers = 8) as executor:
        args1 = range(0, 158000, 1000)
        args2 = range(1000, 159000, 1000)
        args = list(zip(args1, args2))
        args3 = [(158000, 158082)]
        args += args3
        fs = executor.map(get_paths, args)
    finish = time.perf_counter()
    print(f'{finish - start}')

In [ ]:
whole = []
for f in fs:
    whole += f

In [20]:
with open(f'{scope}_{cam}_with_objects.pickle', "wb") as outfile:
    pickle.dump(whole, outfile)

In [21]:
len(whole)

123484

In [ ]:
whole[0]